<a href="https://colab.research.google.com/github/YongWanJin/KDTteam5_semiproject/blob/main/code/%EC%A2%8C%ED%91%9C%EA%B3%84_%EB%B3%80%EA%B2%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 도움을 준 사이트 : https://wooiljeong.github.io/python/transform_coord/

### 좌표계 변환에 필요한 패키지 pyproj 설치
!pip install pyproj

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.3 MB 7.3 MB/s 


In [ ]:
### 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
### colab으로 불러온 파일을 데이터프레임df에 저장 & 필요한 모듈 불러오기
# 사용한 데이터 : 서울시 우리마을가게 상권분석서비스(상권영역).csv

import pandas as pd
import numpy as np
import pyproj
import folium
from IPython.display import display 

df = pd.read_csv("/content/gdrive/MyDrive/datafile/서울시 우리마을가게 상권분석서비스(상권영역).csv",encoding='CP949')
display(df)
print(df.info()) # '엑스좌표_값'과 '와이좌표_값'이 이미 int형으로 되어있음, 따로 전처리해줄 필요 없음

,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,형태정보
0,202112,A,골목상권,2110753,시흥동 은행나무사거리,191934,438813,11545,11545710,NaN
1,202112,A,골목상권,2110427,노원 을지대학교병원,206265,459431,11350,11350611,NaN
2,202112,A,골목상권,2110755,동일여자고등학교,192300,439070,11545,11545680,NaN
3,202112,A,골목상권,2110739,시흥3동 우편취급국,191289,438014,11545,11545690,NaN
4,202112,A,골목상권,2111017,송파나루역 3번,210222,445516,11710,11710562,NaN
...,...,...,...,...,...,...,...,...,...,...
1666,202112,U,관광특구,1001493,동대문패션타운 관광특구,200974,451975,11140,11140590,NaN
1667,202112,U,관광특구,1001494,종로?청계 관광특구,199768,452273,11110,11110615,NaN
1668,202112,U,관광특구,1001495,잠실 관광특구,210191,446339,11710,11710566,NaN
1669,202112,U,관광특구,1001496,강남 마이스 관광특구,205310,445727,11680,11680580,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1671 entries, 0 to 1670
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   기준_년월_코드    1671 non-null   int64  
 1   상권_구분_코드    1671 non-null   object 
 2   상권_구분_코드_명  1671 non-null   object 
 3   상권_코드       1671 non-null   int64  
 4   상권_코드_명     1671 non-null   object 
 5   엑스좌표_값      1671 non-null   int64  
 6   와이좌표_값      1671 non-null   int64  
 7   시군구_코드      1671 non-null   int64  
 8   행정동_코드      1671 non-null   int64  
 9   형태정보        0 non-null      float64
dtypes: float64(1), int64(6), object(3)
memory usage: 130.7+ KB
None


In [ ]:
### 엑스좌표값, 와이좌표값만 추출

coord_x = df[['엑스좌표_값']]
coord_y = df[['와이좌표_값']]
coord = pd.concat([coord_x, coord_y], axis=1)

display(coord)

,엑스좌표_값,와이좌표_값
0,191934,438813
1,206265,459431
2,192300,439070
3,191289,438014
4,210222,445516
...,...,...
1666,200974,451975
1667,199768,452273
1668,210191,446339
1669,205310,445727


In [ ]:
### 원본의 좌표계를 시각화 가능한 좌표계로 변환하기 위한 함수 정의

def coord_transfer(coord, p1_type, p2_type) :

  #coord: x, y 좌표 정보가 담긴 NumPy Array
  #p1_type: 입력 좌표계 정보 ex) 원본 자료의 좌표계 종류는 epsg:5181 라고 함.
  #p2_type: 출력 좌표계 정보 ex) 시각화할때 쓰는 좌표게 종류는 epsg:4326

  p1 = pyproj.Proj(init=p1_type)
  p2 = pyproj.Proj(init=p2_type)
  fx, fy = pyproj.transform(p1, p2, coord[:, 0], coord[:, 1])

  return np.dstack([fx, fy])[0]


In [ ]:
### 좌표계 변환 실시

coord = np.array(coord) # 함수에 적용하기 위해 데이터프레임'coord'의 타입을 np.array로 변환
p1_type = 'epsg:5181'
p2_type = 'epsg:4326'

result = coord_transfer(coord, p1_type, p2_type)
print(result)

In [ ]:
### 변환된 좌표계를 원본 데이터 df에 추가

df['경도'] = result[:,0]
df['위도'] = result[:,1]
display(df)

In [ ]:
### 잘 되나 테스트

sample = df.sample(n=10) # 임의로 10개 뽑아봄
lat_c, lon_c = 37.53165351203043, 126.9974246490573
m = folium.Map(location=[lat_c, lon_c], zoom_start=6)
for _, row in sample.iterrows():
    lat, lon = row['위도'], row['경도']
    folium.Marker(location=[lat, lon]).add_to(m)
m

In [ ]:
### colab에 저장, 다운로드 할수 있는 상태가 됨.

df.to_csv('directory\상권영역_위도경도추가.csv')

# 이제 이 데이터를 가지고 시각화를 할 수 있게됨.